In [1]:
import os
import random

import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

In [2]:
image_dir = '/Users/aim/Downloads/GrayScaleBackground_Dataset-main/TrainSet_fmri'

canvas_size = 400
image_size = 100
pad = 20

positions = {
    'top_left': (pad, pad),
    'top_right': (canvas_size - image_size - pad, pad),
    'bottom_left': (pad, canvas_size - image_size - pad),
    'bottom_right': (canvas_size - image_size - pad, canvas_size - image_size - pad)
}

In [7]:
# load all images in a category
def load_images_by_category(base_dir):
    categories = {}
    for root, dirs, files in os.walk(base_dir):
        for dir_name in dirs:
            cat_path = os.path.join(root, dir_name)
            image_files = [os.path.join(cat_path, f)
                           for f in os.listdir(cat_path)
                           if f.lower().endswith(('.png', '.bmp'))]
            if image_files:
                categories[dir_name] = image_files
    return categories

# Create a stimulus image
def create_stimulus(categories, mode='same_image'):
    canvas = Image.new('RGB', (canvas_size, canvas_size), (128,128,128)) # change background color

    if mode == 'same_image':
        cat = random.choice(list(categories.keys()))
        img_path = random.choice(categories[cat])
        img = Image.open(img_path).resize((image_size, image_size))
        imgs = [img, img.copy()]
    elif mode == 'same_category':
        cat = random.choice(list(categories.keys()))
        img_paths = random.sample(categories[cat], 2)
        imgs = [Image.open(p).resize((image_size, image_size)) for p in img_paths]
    elif mode == 'different_category':
        cat1, cat2 = random.sample(list(categories.keys()), 2)
        img1 = Image.open(random.choice(categories[cat1])).resize((image_size, image_size))
        img2 = Image.open(random.choice(categories[cat2])).resize((image_size, image_size))
        imgs = [img1, img2]
    else:
        raise ValueError(f"Unknown mode: {mode}")

    corners = random.sample(list(positions.values()), 2)
    for img, pos in zip(imgs, corners):
        canvas.paste(img, pos)

    return canvas

In [8]:
categories = load_images_by_category(image_dir)
num_imgs = 100

save_root = './all_stim_gray'
if not os.path.exists(save_root):
    os.makedirs(save_root)

for mode in ['same_image', 'same_category', 'different_category']:
    save_dir = os.path.join(save_root, mode)
    os.makedirs(save_dir, exist_ok=True)
    
    for n in range(num_imgs):
        try:
            stim = create_stimulus(categories, mode=mode)
            stim.save(os.path.join(save_dir, f'stimulus{n:02d}_{mode}.png'))
        except Exception as e:
            print(f"Error generating {mode} stimulus {n}: {e}")

Error generating same_category stimulus 8: Sample larger than population or is negative
